In [1]:
!pip install torch torchvision onnx onnxruntime opencv-python
!git clone https://github.com/JingyunLiang/SwinIR.git



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.5 MB/s eta 0:00:00
Cloning into 'SwinIR'...
remote: Enumerating objects: 333, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 333 (delta 6), reused 2 (delta 2), pack-reused 323 (from 2)
Receiving objects: 100% (333/333), 29.84 MiB | 37.03 MiB/s, done.
Resolving deltas: 100% (119/119), done.


In [2]:
!pip listpip isn

Package                               Version
------------------------------------- -------------------
absl-py                               1.4.0
absolufy-imports                      0.3.1
accelerate                            1.10.1
aiofiles                              24.1.0
aiohappyeyeballs                      2.6.1
aiohttp                               3.12.15
aiosignal                             1.4.0
alabaster                             1.0.0
albucore                              0.0.24
albumentations                        2.0.8
ale-py                                0.11.2
altair                                5.5.0
annotated-types                       0.7.0
antlr4-python3-runtime                4.9.3
anyio                                 4.10.0
anywidget                             0.9.18
argon2-cffi                           25.1.0
argon2-cffi-bindings                  25.1.0
array_record                          0.8.1
arviz                                 0.22.0
astro

In [2]:
!mkdir /content/experiments
!mkdir /content/experiments/pretrained_models

In [3]:
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/002_lightweightSR_DIV2K_s64w8_SwinIR-S_x4.pth -P /content/experiments/pretrained_models

--2025-08-29 12:33:17--  https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/002_lightweightSR_DIV2K_s64w8_SwinIR-S_x4.pth
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/396770997/6bea09c7-c0f6-412c-8430-867c5aba72a2?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-08-29T13%3A30%3A39Z&rscd=attachment%3B+filename%3D002_lightweightSR_DIV2K_s64w8_SwinIR-S_x4.pth&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-08-29T12%3A30%3A22Z&ske=2025-08-29T13%3A30%3A39Z&sks=b&skv=2018-11-09&sig=V2OmJMGEuw1xLj88Y3H2aWJU5onqtQr6M0i0Vsqv3TY%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1NjQ3MTA5NywibmJm

In [4]:
%cd SwinIR

/content/SwinIR


In [5]:

# export_swinir_onnx.py
import torch
import torch.nn as nn
from models.network_swinir import SwinIR

CKPT = "/content/experiments/pretrained_models/002_lightweightSR_DIV2K_s64w8_SwinIR-S_x4.pth"  # put file in this folder
ONNX_OUT = "swinir_s_x4_lightweight.onnx"

# Build model (lightweight SR, x4)
model = SwinIR(
    upscale=4, in_chans=3, img_size=64, window_size=8, img_range=1.,
    depths=[6, 6, 6, 6], embed_dim=60, num_heads=[6, 6, 6, 6],
    mlp_ratio=2, upsampler='pixelshuffledirect', resi_connection='1conv'
)
model.eval()

# Load weights (key is 'params' for this release)
state = torch.load(CKPT, map_location='cpu')
key = 'params' if 'params' in state else 'params_ema' if 'params_ema' in state else None
if key is not None:
    state = state[key]
model.load_state_dict(state, strict=True)

# Dummy input: NCHW in [0,1], H,W must be multiples of window_size (8)
dummy = torch.randn(1, 3, 64, 64)  # export with small valid size

# Export
torch.onnx.export(
    model, dummy, ONNX_OUT,
    input_names=["lr"], output_names=["sr"],
    dynamic_axes={"lr": {2: "h", 3: "w"}, "sr": {2: "H", 3: "W"}},
    opset_version=17, do_constant_folding=True
)
print("Exported:", ONNX_OUT)


/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.12/dist-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/tmp/ipython-input-1012588315.py:28: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. L

Exported: swinir_s_x4_lightweight.onnx


In [ ]:
%cd ../

In [2]:
import cv2
import numpy as np

# Load the image
img = cv2.imread('/content/0006.png', cv2.IMREAD_COLOR)

# Define the scale factor
scale_factor = 4

# Calculate the new dimensions
new_width = img.shape[1] // scale_factor
new_height = img.shape[0] // scale_factor

# Downsample the image using bicubic interpolation
img_lr = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_CUBIC)

# Save the downsampled image
cv2.imwrite('/content/0006_lr.png', img_lr)


True

In [ ]:
import cv2, numpy as np, onnxruntime as ort

onnx_path = "/content/SwinIR/swinir_s_x4_lightweight.onnx"
sess = ort.InferenceSession(onnx_path, providers=["CUDAExecutionProvider", "CPUExecutionProvider"])

def pad_to_multiple(img, m=8):
    h,w = img.shape[:2]
    ph = (m - h % m) % m; pw = (m - w % w) % m
    if ph or pw:
        img = cv2.copyMakeBorder(img, 0, ph, 0, pw, cv2.BORDER_REFLECT)
    return img, ph, pw

def swinir_x4_ort(img_bgr):
    rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB).astype(np.float32)/255.0
    rgb, ph, pw = pad_to_multiple(rgb, 8)
    inp = np.transpose(rgb,(2,0,1))[None]
    out = sess.run(None, {"lr": inp})[0][0]              # CHW, [0,1]
    sr = np.transpose(out,(1,2,0))
    if ph or pw:
        H,W = sr.shape[:2]; sr = sr[:H - ph*4, :W - pw*4]
    sr = np.clip(sr,0,1)
    return cv2.cvtColor((sr*255).astype(np.uint8), cv2.COLOR_RGB2BGR)

# Example
lr = cv2.imread("/content/0006_lr.png")  # low-res image
sr = swinir_x4_ort(lr)
cv2.imwrite("sr_x4.png", sr)
print("Saved sr_x4.png")


Saved sr_x4.png
